In [ ]:
import gc
from itertools import combinations
import pathlib
from typing import Dict, List
import warnings
import yaml

import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import polars as pl
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit

gc.enable()
pd.set_option('display.max_columns', 200)
warnings.simplefilter('ignore')

In [ ]:
def preprocess(df: pd.DataFrame):
    df = pl.from_pandas(df)
    new_features1 = [
        (pl.col('date_id') % 5).alias('day_of_week'),
        (pl.col('imbalance_buy_sell_flag') + 1).alias('imbalance_buy_sell_flag'),
        (pl.col('ask_price') - pl.col('bid_price')).alias('diff_ask_price_bid_price'),
        (pl.col('ask_price') - pl.col('reference_price')).alias('diff_ask_price_reference_price'),
        (pl.col('bid_price') - pl.col('reference_price')).alias('diff_bid_price_reference_price'),
        (pl.col('ask_price') - pl.col('wap')).alias('diff_ask_price_wap'),
        (pl.col('bid_price') - pl.col('wap')).alias('diff_bid_price_wap'),
        (pl.col('far_price') - pl.col('near_price')).alias('diff_far_price_near_price'),
        (pl.col('far_price') - pl.col('reference_price')).alias('diff_far_price_reference_price'),
        (pl.col('near_price') - pl.col('reference_price')).alias('diff_near_price_reference_price'),
        (pl.col('ask_size') - pl.col('bid_size')).alias('diff_ask_size_bid_size'),
        (pl.col('ask_size') - pl.col('matched_size')).alias('diff_ask_size_matched_size'),
        (pl.col('bid_size') - pl.col('matched_size')).alias('diff_bid_size_matched_size'),
        (pl.col('imbalance_size') - pl.col('matched_size')).alias('diff_imbalance_size_matched_size'),
        (pl.col('ask_price') + pl.col('bid_price')).alias('sum_ask_price_bid_price'),
        (pl.col('far_price') + pl.col('near_price')).alias('sum_far_price_near_price'),
        (pl.col('ask_size') + pl.col('bid_size')).alias('sum_ask_size_bid_size'),
        (pl.col('bid_size') * pl.col('ask_price') / (pl.col('bid_size') + pl.col('ask_size'))).alias('wap_factor1'),
        (pl.col('ask_size') * pl.col('bid_price') / (pl.col('bid_size') + pl.col('ask_size'))).alias('wap_factor2'),
        (pl.col('near_price') / pl.col('far_price')).alias('div_near_price_far_price'),

        ((pl.col('ask_price') - pl.col('bid_price')) / (pl.col('reference_price') - pl.col('bid_price'))).alias('feature1'), 
        ((pl.col('ask_price') - pl.col('bid_price')) / (pl.col('wap') - pl.col('bid_price'))).alias('feature2'), 
        ((pl.col('ask_price') - pl.col('reference_price')) / (pl.col('bid_price') - pl.col('reference_price'))).alias('feature3'), 
        ((pl.col('ask_price') - pl.col('reference_price')) / (pl.col('wap') - pl.col('reference_price'))).alias('feature4'), 
        ((pl.col('ask_price') - pl.col('wap')) / (pl.col('bid_price') - pl.col('wap'))).alias('feature5'), 
        ((pl.col('ask_price') - pl.col('wap')) / (pl.col('reference_price') - pl.col('wap'))).alias('feature6'),
        ((pl.col('bid_price') - pl.col('ask_price')) / (pl.col('reference_price') - pl.col('ask_price'))).alias('feature7'),
        ((pl.col('bid_price') - pl.col('ask_price')) / (pl.col('wap') - pl.col('ask_price'))).alias('feature8'),
        ((pl.col('bid_price') - pl.col('reference_price')) / (pl.col('wap') - pl.col('reference_price'))).alias('feature9'),
        ((pl.col('bid_price') - pl.col('wap')) / (pl.col('reference_price') - pl.col('wap'))).alias('feature10'),
        ((pl.col('reference_price') - pl.col('ask_price')) / (pl.col('wap') - pl.col('ask_price'))).alias('feature11'),
        ((pl.col('reference_price') - pl.col('bid_price')) / (pl.col('wap') - pl.col('bid_price'))).alias('feature12'),

        ((pl.col('ask_size') - pl.col('bid_size')) / (pl.col('matched_size') - pl.col('bid_size'))).alias('feature13'), 
        ((pl.col('ask_size') - pl.col('bid_size')) / (pl.col('imbalance_size') - pl.col('bid_size'))).alias('feature14'), 
        ((pl.col('ask_size') - pl.col('matched_size')) / (pl.col('bid_size') - pl.col('imbalance_size'))).alias('feature15'), 
        ((pl.col('ask_size') - pl.col('matched_size')) / (pl.col('imbalance_size') - pl.col('matched_size'))).alias('feature16'), 
        ((pl.col('ask_size') - pl.col('imbalance_size')) / (pl.col('bid_size') - pl.col('imbalance_size'))).alias('feature17'), 
        ((pl.col('ask_size') - pl.col('imbalance_size')) / (pl.col('matched_size') - pl.col('imbalance_size'))).alias('feature18'),
        ((pl.col('bid_size') - pl.col('ask_size')) / (pl.col('matched_size') - pl.col('ask_size'))).alias('feature19'),
        ((pl.col('bid_size') - pl.col('ask_size')) / (pl.col('imbalance_size') - pl.col('ask_size'))).alias('feature20'),
        ((pl.col('bid_size') - pl.col('matched_size')) / (pl.col('imbalance_size') - pl.col('matched_size'))).alias('feature21'),
        ((pl.col('bid_size') - pl.col('imbalance_size')) / (pl.col('matched_size') - pl.col('imbalance_size'))).alias('feature22'),
        ((pl.col('matched_size') - pl.col('ask_size')) / (pl.col('imbalance_size') - pl.col('ask_size'))).alias('feature23'),
        ((pl.col('matched_size') - pl.col('bid_size')) / (pl.col('imbalance_size') - pl.col('bid_size'))).alias('feature24'),

        (pl.col('bid_price') / pl.col('ask_price')).alias('feature25'),
        (pl.col('reference_price') / pl.col('ask_price')).alias('feature26'),
        (pl.col('wap') / pl.col('ask_price')).alias('feature27'),
        (pl.col('reference_price') / pl.col('bid_price')).alias('feature28'),
        (pl.col('wap') / pl.col('bid_price')).alias('feature29'),

        (pl.col('bid_size') / pl.col('ask_size')).alias('feature30'),
        (pl.col('ask_size') / pl.col('matched_size')).alias('feature31'),
        (pl.col('bid_size') / pl.col('matched_size')).alias('feature32'),
        (pl.col('imbalance_size') / pl.col('matched_size')).alias('feature33'),
        (pl.col('ask_size') / pl.col('imbalance_size')).alias('feature34'),
        (pl.col('bid_size') / pl.col('imbalance_size')).alias('feature35'),

        (pl.col('imbalance_size') * (pl.col('ask_price') - pl.col('bid_price'))).alias('feature36'),
        ((pl.col('ask_price') - pl.col('bid_price')) * ((pl.col('ask_size') - pl.col('bid_size')) / (pl.col('ask_size') + pl.col('bid_size')))).alias('feature37'),
    ]

    new_features2 = [
        pl.col('wap').mean().over(['date_id', 'seconds_in_bucket']).alias(f'mean_wap_over_stocks'),
    ]

    lag_features = [
        'ask_price',
        'bid_price',
        'reference_price',
        'wap',
        'far_price',
        'near_price',
        'mean_wap_over_stocks',
        'ask_size',
        'bid_size',
        'matched_size',
        'imbalance_size',
    ]

    new_features3 = [
        # pl.col(lag_features).diff(n=1).over(['stock_id']).name.suffix('_diff1'),
        # pl.col(lag_features).diff(n=2).over(['stock_id']).name.suffix('_diff2'),
        # pl.col(lag_features).diff(n=3).over(['stock_id']).name.suffix('_diff3'),
        # pl.col(lag_features).diff(n=4).over(['stock_id']).name.suffix('_diff4'),
        # pl.col(lag_features).diff(n=5).over(['stock_id']).name.suffix('_diff5'),
        # pl.col(lag_features).diff(n=6).over(['stock_id']).name.suffix('_diff6'),

        pl.col(lag_features).pct_change(n=1).over(['stock_id']).name.suffix('pct_change1'),
        pl.col(lag_features).pct_change(n=2).over(['stock_id']).name.suffix('pct_change2'),
        pl.col(lag_features).pct_change(n=3).over(['stock_id']).name.suffix('pct_change3'),
        pl.col(lag_features).pct_change(n=4).over(['stock_id']).name.suffix('pct_change4'),
        pl.col(lag_features).pct_change(n=5).over(['stock_id']).name.suffix('pct_change5'),
        pl.col(lag_features).pct_change(n=6).over(['stock_id']).name.suffix('pct_change6'),
    ]
    
    transform_features = [
        'imbalance_size',
        'reference_price',
        'matched_size',
        'far_price',
        'near_price',
        'bid_price',
        'bid_size',
        'ask_price',
        'ask_size',
        'wap',
    ]

    new_features4 = [
        ((pl.col(feature) - pl.col(feature).quantile(0.5)) / (pl.col(feature).quantile(0.75) - pl.col(feature).quantile(0.25)))
        .over(['date_id', 'seconds_in_bucket'])
        .alias(f'robust_scaled_{feature}_over_stocks')
        for feature in transform_features
    ]
    
    df = (
        df
        .with_columns(new_features1)
        .with_columns(new_features2)
        .with_columns(new_features3)
        .drop(['row_id'])
        .to_pandas()gra
    )
    return df

In [ ]:
inputs_dir_path = pathlib.Path('../inputs')
outputs_dir_path = pathlib.Path('../outputs')
if not outputs_dir_path.is_dir():
    outputs_dir_path.mkdir()

train_df = pd.read_csv(inputs_dir_path. joinpath('train.csv'))
train_df.drop(columns=['time_id'], inplace=True)
train_df = preprocess(train_df)
train_df = train_df.dropna(subset=['target'])
display(train_df)

## train lightgbm models using cross validation

In [ ]:
def train(
        dataset: pd.DataFrame,
        outputs_dir: pathlib.Path,
    ):
    
    target_columns = ['stock_id', 'date_id', 'seconds_in_bucket', 'target']
    feature_columns = [col for col in dataset.columns if col not in ['date_id', 'target']]
    days= np.arange(dataset['date_id'].min(), dataset['date_id'].max())
    fimps = []
    best_param_records = {}
    best_value_records = {}
    history = {
        'train_mae': [],
        'valid_mae': [],
        'test_mae': [],
    }
    valid_days = days[-60*2:-60]
    test_days = days[-60:]
    valid_X = dataset.query('date_id in @valid_days')[feature_columns]
    valid_y = dataset.query('date_id in @valid_days')[target_columns]
    test_X = dataset.query('date_id in @test_days')[feature_columns]
    test_y = dataset.query('date_id in @test_days')[target_columns]

    train_day_limits = [(300, 360), (240, 360), (180, 360), (120, 360), (60, 360), (0, 360)]

    for k, limits in enumerate(train_day_limits):
        train_days = np.arange(*limits)
        print(f'fold {k+1}')
        print(train_days)
        print(valid_days)
        
        plot_time(days, train_days, valid_days, test_days)
        
        train_X = dataset.query('date_id in @train_days')[feature_columns]
        train_y = dataset.query('date_id in @train_days')[target_columns]
        print(f'train_X.shape: {train_X.shape}, train_y.shape: {train_y.shape}')
        print(f'valid_X.shape: {valid_X.shape}, valid_y.shape: {valid_y.shape}')
        print(f'test_X.shape: {test_X.shape}, test_y.shape: {test_y.shape}')
        
        objective = Objective(train_X, train_y['target'], valid_X, valid_y['target'])
        study = optuna.create_study(sampler=optuna.samplers.TPESampler(seed=42))
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        study.optimize(objective, n_trials=5)

        best_params = study.best_params
        add_params = {
            'task': 'train',
            'boosting_type': 'gbdt',
            'objective': 'mae',
            'metric': 'mae',
            'seed': 42,
            'num_leaves': 256,
        }
        best_params.update(add_params)
        
        [print(f'{k}: {v}') for k, v in best_params.items()]
        print(f'best value: {study.best_value}')

        callbacks = [
            lgb.early_stopping(stopping_rounds=100, verbose=True),
            lgb.log_evaluation(100),
        ]
        
        train_dataset = lgb.Dataset(
            train_X,
            train_y['target'],
            #categorical_feature=['imbalance_buy_sell_flag'],
        )

        valid_dataset = lgb.Dataset(
            valid_X,
            valid_y['target'],
            #categorical_feature=['imbalance_buy_sell_flag'],
        )
        
        model = lgb.train(
            params=best_params,
            train_set=train_dataset,
            valid_sets=[train_dataset, valid_dataset],
            valid_names=['train', 'valid'],
            callbacks=callbacks,
            num_boost_round=3000,
        )
        model.save_model(
            outputs_dir.joinpath(f'lightgbm_optuna_fold{k+1}.txt'),
            num_iteration=model.best_iteration
        )
        
        best_params['num_boost_round'] = model.best_iteration
        best_param_records[f'fold{k+1}'] = best_params
        best_value_records[f'fold{k+1}'] = study.best_value

        fimp = model.feature_importance(importance_type='gain')
        fimp = pd.DataFrame(fimp, index=feature_columns, columns=[f'fold{k+1}'])
        fimps.append(fimp)

        train_pred = model.predict(train_X, num_iteration=model.best_iteration)
        valid_pred = model.predict(valid_X, num_iteration=model.best_iteration)
        test_pred = model.predict(test_X, num_iteration=model.best_iteration)

        test_y[f'regression_fold{k+1}'] = test_pred

        history['train_mae'].append(mean_absolute_error(train_y['target'], train_pred))
        history['valid_mae'].append(mean_absolute_error(valid_y['target'], valid_pred))
        history['test_mae'].append(mean_absolute_error(test_y['target'], test_pred))
        
        del objective, study
        del train_X, train_y, train_dataset, valid_dataset, model, fimp
        del train_pred, valid_pred, test_pred
        gc.collect()

    del valid_X, valid_y, test_X
    gc.collect()

    history = pd.DataFrame.from_dict(history)
    
    fimps = pd.concat(fimps, axis=1)
    mean_fimps = fimps.mean(axis=1)
    std_fimps = fimps.std(axis=1)
    fimps['mean_fimps'] = mean_fimps
    fimps['std_fimps'] = std_fimps
    fimps.sort_values(by='mean_fimps', inplace=True)
    
    test_y['regression'] = test_y[[f'regression_fold{k+1}' for k in range(len(train_day_limits))]].mean(axis=1)
    test_y_mae = mean_absolute_error(test_y['target'], test_y['regression'])
    print(f'test_y mae: {test_y_mae:.4f}')
    
    with open(outputs_dir.joinpath('result_lightgbm_optuna.yaml'), 'w') as f:
        yaml.dump(
            {
                'best_param_records': best_param_records,
                'best_value_records': best_value_records,
                'test_y rmse': test_y_mae,
            },
            f,
            default_flow_style=False
        )
    return history, test_y, fimps, best_param_records


class Objective:
    def __init__(self, train_X, train_y, valid_X, valid_y):
        self.train_X = train_X
        self.train_y = train_y
        self.valid_X = valid_X
        self.valid_y = valid_y

    def __call__(self, trial):
        params = {
            'task': 'train',
            'boosting_type': 'gbdt',
            'objective': 'mae',
            'metric': 'mae',
            'learning_rate': trial.suggest_float('learning_rate', 1e-03, 1e-01),
            'seed': 42,
            'max_depth':  trial.suggest_int('max_depth', 3, 10),
            'num_leaves': 256,
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 100),
            'feature_fraction': trial.suggest_float('feature_fraction_bynode', 0.4, 1.0),
            'feature_fraction_bynode': trial.suggest_float('feature_fraction_bynode', 0.4, 1.0),
            'lambda_l2': trial.suggest_float('lambda_l2', 0.1, 10),
            #'bagging_fraction': 0.6,
            'verbose': -1,
        }
        
        callbacks = [
            lgb.early_stopping(stopping_rounds=100, verbose=True),
            lgb.log_evaluation(0),
        ]
        
        train_dataset = lgb.Dataset(self.train_X, self.train_y)
        valid_dataset = lgb.Dataset(self.valid_X, self.valid_y)

        model = lgb.train(
            params=params,
            train_set=train_dataset,
            valid_sets=[train_dataset, valid_dataset],
            valid_names=['train', 'valid'],
            callbacks=callbacks,
            num_boost_round=3000,
        )

        preds = model.predict(self.valid_X, num_iteration=model.best_iteration)
        mae = mean_absolute_error(preds, self.valid_y)

        del params, callbacks, train_dataset, valid_dataset, model, preds
        gc.collect()

        return mae
    

def plot_time(all_time, train_time, valid_time, test_time):
    _, ax = plt.subplots()
    ax.barh(y='all', height=0.6, width=len(all_time), left=0, color='tab:blue')
    ax.barh(y='train+valid+test', height=0.6, width=[len(train_time), len(valid_time), len(test_time)],
            left=[train_time.min(), valid_time.min(), test_time.min()], color=['tab:orange', 'tab:green', 'tab:red'])
    xcenter = [len(all_time)//2, train_time.min()+len(train_time)//2,
               valid_time.min()+len(valid_time)//2, test_time.min()+len(test_time)//2]
    ycenter = [0, 1, 1, 1]
    width = [f'all\n{len(all_time)}', f'train\n{len(train_time)}', f'valid\n{len(valid_time)}', f'test\n{len(test_time)}']
    for x, y, w in zip(xcenter, ycenter, width):
        ax.text(x, y, str(w),  ha='center', va='center')
    ax.set_xticks([train_time.min(), valid_time.min(), test_time.min(), len(all_time)])
    ax.grid(axis='x', linestyle='--')
    ax.tick_params(axis='x', labelrotation=45)
    plt.show()

In [ ]:
history, result, fimps, best_params_records = train(
    dataset=train_df,
    outputs_dir=outputs_dir_path,
)

In [ ]:
display(result)
print(fimps.shape)
display(fimps.tail(50))

_, ax = plt.subplots(figsize=(12, 18))
fimps['mean_fimps'].plot(kind='barh', xerr=fimps['std_fimps'], capsize=3, ax=ax)  
plt.tight_layout()
plt.show()

fimps_quantile_th = fimps['mean_fimps'].quantile(q=0.2)
display(fimps.query('mean_fimps < @fimps_quantile_th').index)

In [ ]:
history.plot(marker='.', linestyle=':')
plt.show()

In [ ]:
_, ax = plt.subplots()
ax.hist2d(result['regression'], result['target'], bins=100, cmap='Blues', vmax=1e+03)
ax.plot([-100, 100], [-100, 100], color='tab:orange')
ax.set_xlabel('regression')
ax.set_ylabel('target')
plt.show()

r = np.corrcoef(result['regression'], result['target'])
print(f'correlation coeeficient: {r[0, 1]:.4f}')

## train lightgbm model using all data

In [ ]:
best_params = pd.DataFrame(best_params_records)
display(best_params)

new_best_params = {}
for idx in best_params.index:
    if best_params.loc[idx].nunique() > 1:
        new_best_params[idx] = best_params.loc[idx].mean().item()
    else:
        new_best_params[idx] = best_params.loc[idx].unique().item()
    if isinstance(best_params.loc[idx].iloc[0], int):
        new_best_params[idx] = int(new_best_params[idx])

num_boost_round = new_best_params.pop('num_boost_round')
print(new_best_params)

In [ ]:
target_columns = ['stock_id', 'date_id', 'seconds_in_bucket', 'target']
feature_columns = [col for col in train_df.columns if col not in ['date_id', 'target']]

callbacks = [
    lgb.log_evaluation(100),
]

train_dataset = lgb.Dataset(
    train_df[feature_columns],
    train_df[target_columns]['target'],
)

del train_df
gc.collect()

model = lgb.train(
    params=new_best_params,
    train_set=train_dataset,
    callbacks=callbacks,
    num_boost_round=num_boost_round,
)

model.save_model(
    outputs_dir_path.joinpath(f'lightgbm_trained_using_alldata.txt'),
    num_iteration=model.best_iteration
)